In [20]:
import pinocchio as pin
import math
import time
from pinocchio.robot_wrapper import RobotWrapper
from pinocchio.utils import *
import pinocchio as pio
import eigenpy
from scipy.optimize import fmin_bfgs
import numpy as np

import casadi as ca

from pinocchio import casadi as cpin

# from pinocchio.visualize import GepettoVisualizer

import matplotlib.pyplot as plt

In [21]:
exampleRobotDataPath = '/home/n7student/Documents/Boulot/CNRS@CREATE/Codes/OC & IRL/Starting/3 - URDF et pinocchio/assets/'
urdf = exampleRobotDataPath + 'mon_robot.urdf'
robot = RobotWrapper.BuildFromURDF( urdf, [ exampleRobotDataPath, ] )
# robot.setVisualizer(GepettoVisualizer()) # AJOUT
robot.initViewer(loadModel=True)
NQ = robot.model.nq
NV = robot.model.nv

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7004/static/


In [22]:
q = ca.SX.sym("q",NQ,1)
dq = ca.SX.sym("dq",NQ,1)

In [23]:
opti = ca.Opti()

# cadre du problème
# temporel
t_0 = 0
t_f = 1
N = 100  # nombre de points de discrétisation
dt = (t_f - t_0) / N

# longueurs
L_1 = 1
L_2 = 1
N_angles = 2

# variables à optimiser
q = opti.variable(N_angles, N)
dq = opti.variable(N_angles, N)

# déclaration de la matrice de poids
# NOTE : j'avais mis des np.ones au début, mais cela ne marchait pas, il faut essayer de rester avec des objets casadi
w_1 = ca.DM.ones((N_angles, N))
w_2 = ca.DM.ones(N)
w_3 = ca.DM.zeros(N)
w_4 = ca.DM.zeros(N)


# \Phi_1 : coût sur les vitesses angulaires
Phi_1 = dq**2
J_1 = w_1[0,:] @ Phi_1[0,:].T + w_1[1,:] @ Phi_1[1,:].T

# variables symboliques indépendantes pour un pas de temps
# q_sym = ca.MX.sym("q", 2)
# q_sym = ca.SX.sym("q", 2)
q1_sym = ca.SX.sym("q1")
q2_sym = ca.SX.sym("q2")
# dq_sym = ca.MX.sym("dq", 2)
# dq_sym = ca.SX.sym("dq", 2)
dq1_sym = ca.SX.sym("dq1")
dq2_sym = ca.SX.sym("dq2")


# NOTE : les implémentations de Px et gradient sont inexactes, je vais les réimplémenter

"""
# Px et gradient
Px = L_1*ca.cos(q_sym[0]) + L_2*ca.cos(q_sym[0] + q_sym[1])
grad_Px = ca.gradient(Px, q_sym)
dPx_sq_fun = ca.Function("dPx_sq", [q_sym], [ca.dot(grad_Px, grad_Px)])

# Py et gradient
Py = L_1*ca.sin(q_sym[0]) + L_2*ca.sin(q_sym[0] + q_sym[1])
grad_Py = ca.gradient(Py, q_sym)
dPy_sq_fun = ca.Function("dPy_sq", [q_sym], [ca.dot(grad_Py, grad_Py)])
"""


# Réimplémentation de Px et Py
# Px = L_1*ca.cos(q_sym[0]) + L_2*ca.cos(q_sym[0] + q_sym[1])
Px = L_1*ca.cos(q1_sym) + L_2*ca.cos(q1_sym + q2_sym)
Px_fun = ca.Function("Px", [q1_sym, q2_sym], [Px])
# Py = L_1*ca.sin(q_sym[0]) + L_2*ca.sin(q_sym[0] + q_sym[1])
Py = L_1*ca.sin(q1_sym) + L_2*ca.sin(q1_sym + q2_sym)
Py_fun = ca.Function("Py", [q1_sym, q2_sym], [Py])
# grad_Px = ca.gradient(Px, q_sym)
grad_Px = ca.gradient(Px, ca.vertcat(q1_sym, q2_sym))
# grad_Py = ca.gradient(Py, q_sym)
grad_Py = ca.gradient(Py, ca.vertcat(q1_sym, q2_sym))
# dPx_fun = ca.Function("dPx", [q_sym, dq_sym] , [grad_Px.T @ dq_sym])
dPx_fun = ca.Function("dPx", [ca.vertcat(q1_sym, q2_sym), ca.vertcat(dq1_sym, dq2_sym)] , [grad_Px.T @ ca.vertcat(dq1_sym, dq2_sym)])
# dPy_fun = ca.Function("dPy", [q_sym, dq_sym] , [grad_Py.T @ dq_sym])
dPy_fun = ca.Function("dPy", [ca.vertcat(q1_sym, q2_sym), ca.vertcat(dq1_sym, dq2_sym)] , [grad_Py.T @ ca.vertcat(dq1_sym, dq2_sym)])



# q_sym2 = ca.SX.sym("q2", 2)

# f et Jacobien
# f = ca.vertcat(L_1*ca.cos(q_sym[0]) + L_2*ca.cos(q_sym[0] + q_sym[1]),
#             L_1*ca.sin(q_sym[0]) + L_2*ca.sin(q_sym[0] + q_sym[1]))
f = ca.vertcat(L_1*ca.cos(q1_sym) + L_2*ca.cos(q1_sym + q2_sym),
            L_1*ca.sin(q1_sym) + L_2*ca.sin(q1_sym + q2_sym),
            0)
# f = ca.Function("f", [q_sym2], [ca.vertcat(L_1*ca.cos(q_sym2[0]) + L_2*ca.cos(q_sym2[0] + q_sym2[1]),
#             L_1*ca.sin(q_sym2[0]) + L_2*ca.sin(q_sym2[0] + q_sym2[1]))])
# J = ca.jacobian(f, q_sym)
J = ca.jacobian(f, ca.vertcat(q1_sym, q2_sym))
J_func = ca.Function("J_func", [q1_sym, q2_sym], [J])
JTJ = J.T @ J
# JTJ = ca.Function("JJt", [q_sym2], [J.T @ J])
eps = 1e-6
# NOTE : j'avais tenté une autre approche pour le coût suivant, mais on m'a dit de l'abandonner
# log_det_JJt_fun = Function("log_det_JJt", [q_sym2], [-log(fabs(det(JJt) - 1))])
# log_det_JJt_fun = ca.Function("log_det_JJt", [q_sym2], [-ca.log(ca.det(JTJ) + eps)])
# log_det_JJt_fun = ca.Function("log_det_JJt", [q_sym], [-ca.log(ca.det(JTJ) + eps)])
log_det_JJt_fun = ca.Function("log_det_JJt", [ca.vertcat(q1_sym, q2_sym)], [-ca.log(ca.det(JTJ) + eps)])
JJt_fun = ca.Function("JJt", [ca.vertcat(q1_sym, q2_sym)], [JTJ])

J_2 = 0
J_3 = 0
J_4 = 0
for t in range(N):
    q_t = ca.vertcat(q[0, t], q[1, t])
    dq_t = ca.vertcat(dq[0, t], dq[1, t])
    # J_2 += w_2[t] * dPx_sq_fun(q_t)
    J_2 += w_2[t] * ca.sqrt(dPx_fun(q_t, dq_t)**2)
    # J_3 += w_3[t] * dPy_sq_fun(q_t)
    J_3 += w_2[t] * ca.sqrt(dPy_fun(q_t, dq_t)**2)
    J_4 += w_4[t] * log_det_JJt_fun(q_t)

# fonction que l'on cherche à minimiser
opti.minimize(J_1 + J_2 + J_3 + J_4)

# contraintes d’intégration
for t in range(N-1):
    opti.subject_to(q[:, t+1] == q[:, t] + dt * dq[:, t])
opti.subject_to(q[:, 0] == [0, ca.pi/4])
opti.subject_to(L_1*ca.cos(q[0, -1]) + L_2*ca.cos(q[0, -1] + q[1, -1]) == -0.75)

# conditions supplémentaires sur les angles, non demandé dans l'énoncé
# opti.subject_to(opti.bounded(-pi/2, q, pi/2))

opti.solver("ipopt")
sol = opti.solve()

q1 = sol.value(q[0,:])
q2 = sol.value(q[1,:])

dq1 = sol.value(dq[0,:])
dq2 = sol.value(dq[1,:])

print("q1 =", q1)
print("q2 =", q2)
print("dq1 =", dq1)
print("dq2 =", dq2)

This is Ipopt version 3.14.17, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:      598
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:     1000

Total number of variables............................:      400
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:      201
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  0.0000000e+00 2.75e+00 0.00e+00  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

RuntimeError: Error in Opti::solve [OptiNode] at .../casadi/core/optistack.cpp:165:
.../casadi/core/optistack_internal.cpp:1030: Assertion "return_success(accept_limit)" failed:
Solver failed. You may use opti.debug.value to investigate the latest values of variables. return_status is 'Maximum_Iterations_Exceeded'

In [18]:
robot.display(np.array([q1[0], q2[0]]))

time.sleep(2)
for i in range(N):
    robot.display(np.array([q1[i],q2[i]]))
    time.sleep(0.1)

In [12]:
results_angle = np.array([q1, q2])

In [13]:
results_angle.tofile(f"opti_angles_casadi_w1_{w_1[0]}_w2_{w_2[0]}_w3_{w_3[0]}_w4_{w_4[0]}.txt")